# MicroStrateg(P)ython Migration

## Load libraries

In [1]:
import json
from mstrio.object_management import folder
import pandas as pd
from time import sleep
import shutil
import os
from mstrio.connection import Connection
from mstrio.object_management.migration import (
    PackageType,
    Migration,
    PackageConfig,
    PackageSettings
)

from mstrio.object_management.migration.package import (
    Action,
    ImportStatus,
    PackageStatus,
    ValidationStatus,
)


def bld_mig_list_from_sh_fold(conn, folder_id,action= "USE_EXISTING",include_dependents= False ):

    #obj_l = get_folder_obj_l(conn=conn, folder_id=folder_id)
    i_folder = folder.Folder(connection=conn, id=folder_id)
    obj_l = i_folder.get_contents(to_dictionary=True)
    
    obj_det_l=[]
    for obj_d in obj_l:
        if obj_d["type"] == 18:
            obj_det_d = {}
            obj_det_d["id"] = obj_d["target_info"]["id"]
            obj_det_d["type"] = obj_d["target_info"]["type"]
            obj_det_d["action"] = action
            obj_det_d["include_dependents"] = include_dependents
            #obj_det_d["sub_type_bez"] = ObjectSubTypes(obj_d["target_info"]["subtype"]).name
            obj_det_l.append(obj_det_d.copy())

    return obj_det_l


## Configure Migration

In [2]:
#define Package
usr_migration_folder_name="Daniel_Migration"
usr_package_folder_name="2024123_Demo"


short_cut_folder_id="986E6D1A427030A710A0E68F74290482"

# Connect Projects

source_username="Administrator"
source_password=""
source_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
source_base_url="http://00.000.00.00:8080:8080/MicroStrategyLibrary"

target_username="Administrator"
target_password=""
target_project_id="00F3103E41FF3743A057C3B5313595B7"
target_base_url="http://00.000.00.00:8080/MicroStrategyLibrary"

package_settings = PackageSettings(
    Action.FORCE_REPLACE,
    PackageSettings.UpdateSchema.RECAL_TABLE_LOGICAL_SIZE,
    PackageSettings.AclOnReplacingObjects.REPLACE,
    PackageSettings.AclOnNewObjects.KEEP_ACL_AS_SOURCE_OBJECT,
)

## Connect to projects

In [3]:
source_conn = Connection(base_url=source_base_url,username=source_username,
                  password=source_password,project_id=source_project_id)
#source_conn.headers['Content-type'] = "application/json"

target_conn = Connection(base_url=target_base_url,username=target_username,
                  password=target_password,project_id=target_project_id)

package_name=usr_package_folder_name
mstr_mig_folder=source_conn.environment.storage_service.location 
migration_folder= source_conn.environment.storage_service.location  +"\\"+ usr_migration_folder_name +"\\"+ usr_package_folder_name + "\\"
migration_folder


Connection to MicroStrategy Intelligence Server has been established.
Connection to MicroStrategy Intelligence Server has been established.


'D:\\shared_drive\\MSTR_Migrations\\Daniel_Migration\\2024123_Demo\\'

## Create package

In [4]:

package_content_info=bld_mig_list_from_sh_fold(conn=source_conn, folder_id=short_cut_folder_id,action= "FORCE_REPLACE",include_dependents= False)
package_config = PackageConfig(
    package_settings,  package_content_info # [ package_content_from_object]
)

my_obj_mig = Migration.create_object_migration(
    connection=source_conn,
    toc_view=package_config,
    name=package_name,
    project_id=source_conn.project_id,
)

while my_obj_mig.package_info.status == PackageStatus.CREATING:
    sleep(2)
    my_obj_mig.fetch()

my_obj_mig.package_info.status

Folder object named: '2024123_Demo' with ID: '986E6D1A427030A710A0E68F74290482'
Successfully started creation of migration object with ID: '06E4DA19FD6046C4BD87CDB1072BF8FB:8BA99F7FC4604C539A73D4ADDBCB4B9F'


PackageStatus.CREATED

## Vaildate Package

In [5]:
# Check migration validation status
my_obj_mig.trigger_validation(
    target_env=target_conn, target_project_name=target_conn.project_name
)

while my_obj_mig.validation.status == ValidationStatus.VALIDATING:
    sleep(2)
    my_obj_mig.fetch()


print(my_obj_mig.validation)

Successfully triggered validation process for migration with ID: '06E4DA19FD6046C4BD87CDB1072BF8FB:8BA99F7FC4604C539A73D4ADDBCB4B9F'
Validation(status=ValidationStatus.VALIDATED, progress=100.0, message='Success', creation_date=datetime.datetime(2024, 12, 22, 17, 40, 25, tzinfo=datetime.timezone.utc), last_update_date=None)


## Move Migration Pack to migration folder

In [6]:
package_file = mstr_mig_folder +"/" +my_obj_mig.package_info.storage.path
package_file=os.path.normpath(package_file)
my_obj_mig_path=shutil.move(package_file, migration_folder)

print("org_file: " +package_file)
print("migration folder: " +my_obj_mig_path)

org_file: D:\shared_drive\MSTR_Migrations\mstr\shared\migrations\packages\2024-12-22\92207ABE1D7246B4A9774BB677987FE3.mmp
migration folder: D:\shared_drive\MSTR_Migrations\Daniel_Migration\2024123_Demo\92207ABE1D7246B4A9774BB677987FE3.mmp



## Trigger migration

In [7]:
from_file_mig = Migration.migrate_from_file(
    connection=source_conn,
    file_path=my_obj_mig_path,
    package_type=PackageType.OBJECT,
    name="object_mig_from_local_storage",
    target_project_name=target_conn.project_name,
)

while from_file_mig.import_info.status == ImportStatus.IMPORTING:
    sleep(2)
    from_file_mig.fetch()

from_file_mig.import_info.status

Successfully started migration from file with ID: 'F02F806CFC4B4FEEB22D9CA91AB4B2F6:44DF6E0D101A4D46861B35F85BB0C122'.


ImportStatus.IMPORTED

## Archive Migration

### move undo mmp file to migration folder

In [8]:
## move undo mmp to migration folder
undo_source_file = mstr_mig_folder +"/" +from_file_mig.import_info.undo_storage.path
package_file=os.path.normpath(package_file)
new_undo_file=shutil.move(undo_source_file, migration_folder+"Undo.mmp")

print("org_undo_file: " +undo_source_file)
print("migration folder: " +new_undo_file)

org_undo_file: D:\shared_drive\MSTR_Migrations/mstr/shared/migrations/undo/F02F806CFC4B4FEEB22D9CA91AB4B2F6/44DF6E0D101A4D46861B35F85BB0C122.mmp
migration folder: D:\shared_drive\MSTR_Migrations\Daniel_Migration\2024123_Demo\Undo.mmp


### store migrated objects to CSV file

In [9]:
package_content_df=pd.DataFrame(package_content_info)
package_content_df.to_csv(migration_folder+"package_content.csv")
package_content_df

,id,type,action,include_dependents
0,FBA328574EC126D4A38B51AFEF5D7856,4,FORCE_REPLACE,False
1,ACB99295488C76AA566DADAE2EC30FC5,4,FORCE_REPLACE,False


### save package & migration files

In [10]:
file_path=migration_folder+"package_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(my_obj_mig.package_info))

file_path=migration_folder+"migration_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(from_file_mig.import_info))
